In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
import os
SMITHERY_TOKEN = os.getenv("SMITHERY_API_KEY")

In [37]:
import asyncio
import base64
from langchain_mcp_adapters.client import MultiServerMCPClient

In [38]:
api_key = os.getenv("SMITHERY_API_KEY")  # 또는 직접 문자열 입력
server_path = "@jmanek/google-news-trends-mcp"
config = {}  # 대부분의 경우 빈 config 허용
config_b64 = base64.urlsafe_b64encode(str(config).encode()).decode()

mcp_url = (
    f"https://server.smithery.ai/{server_path}/mcp"
    f"?config={config_b64}&api_key={api_key}"
)

In [39]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "sequential": {
        "url": mcp_url,  # 위에서 구성한 전체 URL
        "transport": "streamable_http"
    }
}) 

In [45]:
print("[🔗 연결 시도] get_tools 실행...")
tools = await client.get_tools()  # 노트북에서는 await 바로 사용
print("[✅ 성공] 도구 목록:")
for t in tools:
    print(" -", t.name)

[🔗 연결 시도] get_tools 실행...
[✅ 성공] 도구 목록:
 - get_news_by_keyword
 - get_news_by_location
 - get_news_by_topic
 - get_top_news
 - get_trending_terms


In [49]:
top_news_tool = next((t for t in tools if t.name == "get_top_news"), None)
print(top_news_tool)
if top_news_tool:
    print("[⚙ 실행] get_top_news")
    result = await top_news_tool.ainvoke({
        "max_results": 5,
        "period": 3,
        "full_data": True,
        "summarize": False
    })
    print(result)
else:
    print("❌ get_top_news 도구를 찾지 못했습니다.")


name='get_top_news' description='\n    Get top news stories from Google News.\n    ' args_schema={'properties': {'period': {'default': 3, 'description': 'Number of days to look back for top articles.', 'minimum': 1, 'title': 'Period', 'type': 'integer'}, 'max_results': {'default': 10, 'description': 'Maximum number of results to return.', 'minimum': 1, 'title': 'Max Results', 'type': 'integer'}, 'full_data': {'default': False, 'description': 'Return full data for each article. If False a summary should be created by setting the summarize flag', 'title': 'Full Data', 'type': 'boolean'}, 'summarize': {'default': True, 'description': 'Generate a summary of the article, will first try LLM Sampling but if unavailable will use nlp', 'title': 'Summarize', 'type': 'boolean'}}, 'type': 'object'} response_format='content_and_artifact' coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x000001B0D7349DA0>
[⚙ 실행] get_top_news
[{"title":"300 South Koreans detained at Hyund